In [1]:
import pandas as pd
import numpy as np

In [2]:
# read data set, show first five rows
tb = pd.read_csv('../Original Data/tb.csv')
tb.head()

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# replace nan values with 0 
tb.fillna(0, inplace=True)

# change format of column names
cols = list(tb.columns[:3]) + [x[7:] for x in tb.columns[3:]]
tb.columns = cols
tb.head()

,iso2,year,new_sp,m04,m514,m014,m1524,m2534,m3544,m4554,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
0,AD,1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AD,1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AD,1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AD,1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AD,1993,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# drop undefined or unnecessary columns
tb.drop('new_sp', axis='columns', inplace=True)
tb.drop('m04', axis='columns', inplace=True)
tb.drop('f04', axis='columns', inplace=True)
tb.drop('m514', axis='columns', inplace=True)
tb.drop('f514', axis='columns', inplace=True)
tb.drop('mu', axis='columns', inplace=True)
tb.drop('fu', axis='columns', inplace=True)

In [5]:
# melt data frame for better transparency
m_tb = tb.melt(id_vars=["iso2","year"],var_name=["demographic"],value_name="cases")


# separation 'demographic' column into 'sex' and 'age' columns
n_s = ['Male' if x == 'm' else 'Female' for x in m_tb['demographic'].str[0]]
m_tb['sex'] = n_s
m_tb['age'] = m_tb['demographic'].str[1:]

# drop analysed column
m_tb.drop('demographic', axis='columns', inplace=True)

# drop rows with undefined country
index_list = m_tb.query("iso2 == 0").index.tolist()
m_tb_2 = m_tb.drop(index=index_list, axis=0)

In [6]:
# change designation in 'age' column
n_tb = m_tb_2.replace(to_replace =["014","1524","2534", "3544","4554","5564", "65"],value =["0-14","15-24","25-34","35-44","45-54","55-64","65+"])

In [7]:
# create pivot table for better transparency and show last 30 rows
p_table = pd.pivot_table(n_tb, values='cases', index=['iso2', 'year', 'age', 'sex'])
p_table.tail(30)

cases
iso2 year age   sex           
ZW   2006 65+   Female    96.0
                Male     199.0
     2007 0-14  Female   185.0
                Male     138.0
          15-24 Female   739.0
                Male     500.0
          25-34 Female  3311.0
                Male    3693.0
          35-44 Female     0.0
                Male       0.0
          45-54 Female   553.0
                Male     716.0
          55-64 Female   213.0
                Male     292.0
          65+   Female    90.0
                Male     153.0
     2008 0-14  Female   145.0
                Male     127.0
          15-24 Female   840.0
                Male     614.0
          25-34 Female     0.0
                Male       0.0
          35-44 Female  2890.0
                Male    3316.0
          45-54 Female   467.0
                Male     704.0
          55-64 Female   174.0
                Male     263.0
          65+   Female   105.0
                Male     185.0

In [8]:
# save analysed data
p_table.to_csv("../Analysis Data/tb_analysed.csv")